# Differentiable Physics VI: Compressible Navier Stokes


We limit the scope of the solver with the following constraints:

- We use the conservative form of the equations, meaning we are looking at a volume which is fixed in space and the fluid is moving through it.
- We use a 2D grid based approach, where each cell in the grid represents a volume element and has size $\Delta x \times \Delta y$.

The following variables are used:

- $\vec{v_{i,j}}$ : velocity with x / y components $u$ and $v$
- $\rho$ : density
- $p$ : pressure
- $E$ : Total energy
- $e$ : internal energy
- $T$ : Temperature
- $\mu$ : viscosity constant
- $k$ : thermal conductivity constant
- $c_p$ : specific heat at constant pressure
- $c_v$ : specific heat at constant volume
- $\gamma$ : ratio of specific heats $c_p / c_v$
- $\Delta t$ : time step size
- $\Delta x$ : grid cell size in x direction
- $\Delta y$ : grid cell size in y direction

We can formulate the navier stokes equations for compressible flow as follows:

$$\frac{U}{\Delta t} + \frac{E}{\Delta x} + \frac{F}{\Delta y} = 0$$

where the state vector $U$ and the two fluxes $E$ and $F$ are defined as:

$$
U = \begin{bmatrix} \rho \\ \rho u \\ \rho v \\ E \end{bmatrix} \quad
E = \begin{bmatrix} \rho u \\ \rho u^2 + p - \tau_{xx} \\ \rho u v - \tau_{xy} \\ (E + p) u - u \tau_{xx} - v \tau_{xy} + q_x \end{bmatrix} 
F = \begin{bmatrix} \rho v \\ \rho u v - \tau_{xy} \\ \rho v^2 + p - \tau_{yy} \\ (E + p) v - u \tau_{xy} - v \tau_{yy} + q_y \end{bmatrix} $$

$\tau_{xx}$, $\tau_{xy}$, $\tau_{yy}$ are the components of the viscous stress tensor:

$$ \tau_{xx} = \mu \left( 2 \frac{\partial u}{\partial x} - \frac{2}{3} \left( \frac{\partial u}{\partial x} + \frac{\partial v}{\partial y} \right) \right) \quad
\tau_{xy} = \mu \left( \frac{\partial u}{\partial y} + \frac{\partial v}{\partial x} \right) \quad
\tau_{yy} = \mu \left( 2 \frac{\partial v}{\partial y} - \frac{2}{3} \left( \frac{\partial u}{\partial x} + \frac{\partial v}{\partial y} \right) \right) $$

and $q_x$, $q_y$ are components of the heat flux vector.

$$ q_x = - \lambda \frac{\partial T}{\partial x} \quad
q_y = - \lambda \frac{\partial T}{\partial y} $$


We now insert the navier stokes equations into the mac cormack method. We start with the predictor step:

$$\bar{U}_{i,j}^{n+1} = U_{i,j}^n - \frac{\Delta t}{\Delta x} \left( E_{i+1, j}^{n} - E_{i, j}^{n} \right) - \frac{\Delta t}{\Delta y} \left( F_{i, j+1}^{n} - F_{i, j}^{n} \right)$$

and the corrector step:

$$U_{i,j}^{n+1} = \frac{1}{2} \left[ \left( U_{i,j}^n + \bar{U}_{i,j}^{n+1} \right) - \frac{\Delta t}{ \Delta x} \left( \bar{E}_{i, j}^{n+1} - \bar{E}_{i-1, j}^{n+1} \right) - \frac{\Delta t}{\Delta y} \left( \bar{F}_{i, j}^{n+1} - \bar{F}_{i, j-1}^{n+1} \right) \right]$$

In [2]:
import jax.numpy as jnp
from jax import value_and_grad, jit
from functools import partial

@partial(jit, static_argnames=("delta"))
def fd(f : jnp.ndarray, delta : float, axis : int):
    return jnp.roll(f, -1, axis=axis) - f / delta

@partial(jit, static_argnames=("delta"))
def bd(f : jnp.ndarray, delta : float, axis : int):
    return f - jnp.roll(f, 1, axis=axis) / delta

@partial(jit, static_argnames=("delta"))
def cf(f : jnp.ndarray, delta : float, axis : int):
    return (jnp.roll(f, -1, axis=axis) - jnp.roll(f, 1, axis=0)) / (2*delta)

In [9]:
# U : [rho, rho*u, rho*v, E_total]
@partial(jit, static_argnames=("dx", "dy", "mu", "c_v", "c_p", "R", "k"))
def unpack(
    U : jnp.ndarray, # state vector
    dx : float, # delta x
    dy : float, # delta y
    mu : float, # viscosity
    c_v : float, # specific heat constant volume
    c_p : float, # specific heat constant pressure
    R : float, # gas constant
    k : float, # thermal conductivity
) -> tuple[jnp.ndarray, jnp.ndarray]:
    """"
    Solver step for the compressible Stokes equations in 2D using the finite difference method along with the Mac Cormack method.
    """
    # unpack state vector
    gamma = c_p / c_v # ratio of specific heats
    rho = U[0] # density
    u = U[1]/rho # velocity in x direction
    v = U[2]/rho # velocity in y direction
    e = U[3]/U[1] - 0.5*(u**2 + v**2) # internal energy
    T = (gamma - 1)*e / R # temperature
    p = (gamma - 1)*rho*e # pressure

    # viscous stress tensors
    tau_xx = 2/3*mu*(2*fd(u, dx, axis=0) - fd(v, dy, axis=1))
    tau_yy = 2/3*mu*(2*fd(v, dy, axis=1) - fd(u, dx, axis=0))
    tau_xy = mu*(fd(u, dx, axis=0) + fd(v, dy, axis=1))

    # heat flux
    q_x = -k*fd(T, dx, axis=0)
    q_y = -k*fd(T, dy, axis=1)

    E = jnp.zeros_like(U)
    F = jnp.zeros_like(U)

    E = E.at[:,:,0].set(rho * u)
    E = E.at[:,:,1].set(rho * u ** 2 + p - tau_xx)
    E = E.at[:,:,2].set(rho * u * v - tau_xy)
    E = E.at[:,:,3].set((U[3] + p) * u - u * tau_xx - v * tau_xy + q_x)

    F = F.at[:,:,0].set(rho * v)
    F = F.at[:,:,1].set(rho * u * v - tau_xy)
    F = F.at[:,:,2].set(rho * v ** 2 + p - tau_yy)
    F = F.at[:,:,3].set((U[3] + p) * v - u*tau_xy - v * tau_yy + q_y)

    return E, F

In [11]:
@partial(jit, static_argnames=("dt", "dx", "dy", "mu", "c_v", "c_p", "R", "k"))
def mac_cormack_2d(
    U : jnp.ndarray, # state vector
    dt : float, # delta time
    dx : float, # delta x
    dy : float, # delta y
    mu : float, # viscosity
    c_v : float, # specific heat constant volume
    c_p : float, # specific heat constant pressure
    R : float, # gas constant
    k : float, # thermal conductivity
) -> jnp.ndarray:

    E, F = unpack(U, dx, dy, mu, c_v, c_p, R, k)

    U_pred = U - \
        dt * fd(E, delta = dx, axis = 0) - \
        dt * fd(F, delta = dy, axis = 1)

    U_fwd = 1/2 *( (U + U_pred) - \
        dt * bd(E, delta = dx, axis = 0) - \
        dt * bd(F, delta = dy, axis = 1) )

    return U_fwd